In [1]:
from importlib import reload 
import sys
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, spearmanr, kendalltau
import plotly.express as px
import matplotlib.pyplot as plt
# adding Folder_2 to the system path
#sys.path.insert(0, r'C:\Users\tomer\Desktop\year3\sem B\workshop_microbiome\code')
sys.path.insert(0, r'C:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\code')

from imports import  *

$$f(D,\Delta_t) = \alpha e^{-\lambda \Delta_t} \cdot \cos\left(\frac{2 \pi \Delta_t}{365}\right) D_{t-1} + \left(1 - \alpha e^{-\lambda \Delta_t} \cdot \cos\left(\frac{2 \pi \Delta_t}{365}\right)\right) \cdot \overline{\{D_1,...,D_{t-2}\}}$$

In [2]:
import model_class
reload(model_class)

<module 'model_class' from 'c:\\Users\\yuvald\\Documents\\Uni\\סמסטר ב\\workshop_microbiome\\Milestone2\\model_class.py'>

In [3]:
data_path = r"C:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\train_data.csv"
metadata_path = r"C:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\train_metadata.csv"

In [13]:
data_df = pd.read_csv(r"C:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\train_data.csv", index_col="sample")
metadata_df = pd.read_csv(r"C:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\train_metadata.csv",  index_col="sample")

In [4]:
super_model = model_class.superModel(data_path, metadata_path)

In [5]:
baboon_101 = super_model.baboons[0]

In [6]:
baboon_101.fit(lambda_=0)

1


ValueError: Input vector should be 1-D.

In [86]:
df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
print(df2)
df1 = df1.sort_values(by='A', ascending=False)
df2 = df2.loc[df1.index]

   A  B
0  1  4
1  2  5
2  3  6


In [85]:
df2

,A,B
2,3,6
1,2,5
0,1,4


In [19]:
### mean of previous histogram
data_df = data_df.copy()
metadata_df = metadata_df.copy()

metadata_df["collection_date"] = (pd.to_datetime(metadata_df['collection_date']) - pd.Timestamp('1970-01-01')).dt.days
microbiom_with_sample_data = data_df.join(metadata_df[["baboon_id", "collection_date"]], how="inner")
microbiom_with_sample_data = microbiom_with_sample_data.sort_values(["baboon_id", "collection_date"])


In [38]:
baboon_data_df = microbiom_with_sample_data[microbiom_with_sample_data["baboon_id"] == "Baboon_101"]

In [40]:

baboon_data_df.loc[:, ~baboon_data_df.columns.isin(['baboon_id','collection_date'])]

x = clr_normalization(baboon_data_df.loc[:, ~baboon_data_df.columns.isin(['baboon_id','collection_date'])])
df_cumulative_mean = x.expanding().mean()

In [42]:
baboon_data_df["delta_t"] = baboon_data_df["collection_date"].diff()

C:\Users\yuvald\AppData\Local\Temp\ipykernel_41284\2478969720.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baboon_data_df["delta_t"] = baboon_data_df["collection_date"].diff()


In [50]:
delta_t = baboon_data_df["collection_date"].diff()

In [56]:
len(delta_t)

53

In [62]:
cos = np.cos((2*np.pi*delta_t[2:].values)/365)

In [60]:
exp = np.exp(-0.5*delta_t[2:].values)

In [63]:
D_t1 = x[1:-1].values

In [65]:
exp

array([1.35335283e-01, 1.71390843e-15, 3.01973834e-02, 1.11089965e-02,
       3.67879441e-01, 6.06530660e-01, 6.82560338e-08, 1.90398028e-17,
       1.25015287e-09, 2.75364493e-05, 3.67879441e-01, 1.37095909e-06,
       3.67879441e-01, 6.82560338e-08, 4.97870684e-02, 1.85539136e-07,
       6.73794700e-03, 2.03468369e-04, 1.05306174e-20, 4.13993772e-08,
       5.04347663e-07, 2.47875218e-03, 4.97870684e-02, 1.26641655e-14,
       3.35462628e-04, 5.10908903e-12, 6.30511676e-16, 1.58321429e-23,
       4.65888615e-15, 3.44247711e-14, 1.90398028e-17, 1.73620528e-20,
       1.60381089e-28, 3.77513454e-11, 4.13993772e-08, 7.26843938e-71,
       2.82077009e-53, 1.83156389e-02, 1.26641655e-14, 7.98490425e-30,
       1.54281120e-13, 8.20849986e-02, 1.13999185e-12, 2.03468369e-04,
       6.82560338e-08, 8.42346375e-12, 7.48518299e-05, 8.20849986e-02,
       4.97870684e-02, 8.20849986e-02, 2.03468369e-04])

In [72]:
from scipy.special import softmax


In [78]:

softmax((exp*cos*D_t1.T).T, axis = 1).sum()

51.0